In [1]:
# Import and set up
from datetime import date
import thetadata.client
from thetadata import (
    Quote,
    StreamMsg,
    ThetaClient,
    OptionRight,
    StreamMsgType
)
from thetadata import StreamResponseType as ResponseType

In [2]:
# Create few variables to grab the data
last_call_quote = Quote()
last_put_quote = Quote()
price = 0

In [3]:

def callback_straddle(msg):
    
    if msg.type != StreamMsgType.QUOTE:
        return
    
    global price
    if msg.contract.isCall:
        last_call_quote.copy_from(msg.quote)
    else:
        last_put_quote.copy_from(msg.quote)
    straddle_bid = round(last_call_quote.bid_price + last_put_quote.bid_price, 2)
    straddle_ask = round(last_call_quote.ask_price + last_put_quote.ask_price, 2)
    straddle_mid = round((straddle_bid + straddle_ask) / 2, 2)
    
    time_stamp = thetadata.client.ms_to_time(
        msg.quote.ms_of_day
    )
    if price != straddle_mid:
        print(
            f"time: {time_stamp} bid: {straddle_bid} mid: {straddle_mid} ask: {straddle_ask}"
        )
        price = straddle_mid

In [4]:
# Next, connect to ThetaData’s servers, start the stream for the call an put, and check the subscription status.

def streaming_straddle():
    client = ThetaClient(
        username="",
        passwd=""
    )
    client.connect_stream(
        callback_straddle
    )
    
    req_id_call = client.req_quote_stream_opt(
        "SPY", date(2023, 5, 5), 410, OptionRight.CALL
    )  # Request quote updates
    
    req_id_put = client.req_quote_stream_opt(
        "SPY", date(2023, 5, 5), 410, OptionRight.PUT
    )
    if (
        client.verify(req_id_call) != ResponseType.SUBSCRIBED
        or client.verify(req_id_put) != ResponseType.SUBSCRIBED
    ):
        raise Exception(
            "Unable to stream contract. A standard/PRO subscription required."
        )



In [5]:
streaming_straddle()

If you require API support, feel free to join our discord server! http://discord.thetadata.us


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



ConnectionError: Unable to connect to the local Theta Terminal Stream process. Try restarting your system.